# SentenceTransformers SBERT

https://www.sbert.net/index.html


## getting started

In [6]:
import numpy as np
import pandas as pd
import pickle

from sentence_transformers import SentenceTransformer, util

In [7]:
## Mult-lingual model that supports Dutch
modelML = SentenceTransformer('xlm-r-100langs-bert-base-nli-mean-tokens')

## import kamer speeches

In [ ]:
## put data chuncks in one zip file
!cat data/CorpusTweedeKamer* > CorpusTweedeKamer.zip

In [2]:
%%time
tweede_kamer = pd.read_csv("CorpusTweedeKamer.zip")
tweede_kamer = (
    tweede_kamer
    .assign(datum = pd.to_datetime(tweede_kamer.date))
    .assign(speaker = tweede_kamer.speaker.str.lower())
)

CPU times: user 13.5 s, sys: 1.09 s, total: 14.6 s
Wall time: 14.9 s


In [8]:
tweede_kamer.shape

(1143366, 12)

In [4]:
recente_speeches = (
    tweede_kamer
    .dropna(subset = ["text"])
    .query('terms > 3')
    .query('terms < 800')
    .query('datum > "2016-01-01"')
)

In [9]:
recente_speeches.shape

(216470, 12)

It takes hours to embed 100K texts on a comouter without GPU. you can spin up a AI VM machine on GCP to speed thing sup



In [ ]:
## if you have gcloud CLI setup: here is code to spin up a machine....
export IMAGE_FAMILY="pytorch-latest-gpu"
export ZONE="europe-west4-a"
export INSTANCE_NAME="lhl-gpu-instance-2"
export INSTANCE_TYPE="n1-standard-8"

gcloud compute instances create $INSTANCE_NAME \
        --zone=$ZONE \
        --image-family=$IMAGE_FAMILY \
        --image-project=deeplearning-platform-release \
        --maintenance-policy=TERMINATE \
        --accelerator="type=nvidia-tesla-v100,count=1" \
        --machine-type=$INSTANCE_TYPE \
        --boot-disk-size=120GB \
        --metadata="install-nvidia-driver=True" \
        --preemptible

# Dont forget to install the right version of torch in our case with the GCP machine we need the cuda 10.1 version
pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# and the package sentence-transformers
pip install -U sentence-transformers

# om de graphics card in actie te zien
watch -d -n 0.5 nvidia-smi 

With a Tesla V100 card it takes around 5m:36s to embed 216470 texts, without it, it will take hours......

In [10]:
speeches = recente_speeches.text.str.lower().values.tolist()

In [77]:
%%time 
sentence_embeddings = modelML.encode(speeches)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


In [62]:
sentence_embeddings.shape

(95314, 768)

In [12]:
#Store sentences & embeddings on disc
with open('kamer_BERT_embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': speeches, 'embeddings': sentence_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)


## use pre computed embedings

In [11]:
#Load sentences & embeddings from disc
with open('kamer_BERT_embeddings_216K.pck', "rb") as fIn:
    stored_data = pickle.load(fIn)
    speeches = stored_data['sentences']
    sentence_embeddings = stored_data['embeddings']

In [12]:
len(speeches)

216470

In [13]:
sentence_embeddings.shape

(216470, 768)

### Voer een semantic search uit

In [14]:
tt = """
We zijn een klein land van 17 miljoen mensen, dat groot is geworden door
samenwerking in onze gezamenlijke strijd tegen het water, door onze handelsgeest
en onze openheid naar de wereld en door onze gemeenschapszin, die mensen
hier een veilig thuis bood. Ons land is al eeuwenlang een plek waar mensen
met verschillende religieuze en culturele achtergronden en botsende politieke
overtuigingen vreedzaam met elkaar samenleven. Waar democratie en recht altijd
het laatste woord hebben.
Het zijn precies die waarden en tradities die nu onder druk staan. De crises van
de afgelopen jaren hebben veel Nederlanders bezorgd en onzeker gemaakt, door
het verlies aan zekerheid, de vrees voor terreur en een toenemend gebrek aan
respect en saamhorigheid. In de ik-samenleving van vandaag hebben we onvoldoende besef van het belang van gedeelde waarden en de onderlinge spelregels
die nodig zijn om met elkaar samen te leven. We zijn te veel gaan geloven in een
vrijheid zonder verantwoordelijkheid, in een wereld van rechten zonder plichten.
De voorbeelden zien we dagelijks om ons heen. Op internet en social media is de
vrijheid van meningsuiting voor sommigen vooral een vrijbrief voor beledigen en
kwetsen. Op straat tarten en treiteren jongeren uit verveling toevallige passanten
en agenten. Maar het gebrek aan waarden en normen zien we ook bij roekeloze
bankiers, sjoemelende multinationals of graaiende bestuurders, die het eigenbelang
boven het algemeen belang stelden. Of bij politici die bewust te grote beloftes doen
om vier jaar later doodleuk sorry te zeggen.
Ondertussen roept de moeizame integratie van 
"""

In [28]:
tt = "ik ga nu naar huis"

In [29]:
query_embedding = modelML.encode(tt, convert_to_tensor=True)
topmatches = util.semantic_search(query_embedding, sentence_embeddings)
topmatches

[[{'corpus_id': 148199, 'score': 0.9213277},
  {'corpus_id': 208131, 'score': 0.8925456},
  {'corpus_id': 209801, 'score': 0.87854594},
  {'corpus_id': 205894, 'score': 0.8723681},
  {'corpus_id': 210261, 'score': 0.8723681},
  {'corpus_id': 176207, 'score': 0.86704165},
  {'corpus_id': 24293, 'score': 0.86661553},
  {'corpus_id': 24962, 'score': 0.86431146},
  {'corpus_id': 128805, 'score': 0.86112165},
  {'corpus_id': 192826, 'score': 0.86062646}]]

In [30]:
top_textst = [ idx['corpus_id']  for idx in topmatches[0]]
[speeches[i] for i in top_textst]

['dan ga ik naar mijn plek.',
 'wel even naar huis geweest!',
 'nou, dan ga ik weer!',
 'dan ben ik erdoorheen.',
 'dan ben ik erdoorheen.',
 'nee, maar ... ik ga weer.',
 'tot slot, mevrouw gesthuizen.',
 'dan ga ik door.',
 'ja, dan ga ik verder en dan rond ik af.',
 'ik ga zo meteen wel.']

In [31]:
recente_speeches.iloc[top_textst]

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,datum
1068912,2018-07-05,NaN,1032,de heer kwint,SP,1363.0,False,6,Dan ga ik naar mijn plek.,NL-TweedeKamer,NLD,2018-07-05
1134233,2019-06-06,NaN,141,minister rutte,NaN,NaN,False,5,Wel even naar huis geweest!,NL-TweedeKamer,NLD,2019-06-06
1136063,2019-06-18,NaN,3,minister rutte,NaN,NaN,False,5,"Nou, dan ga ik weer!",NL-TweedeKamer,NLD,2019-06-18
1131792,2019-05-22,NaN,288,minister van nieuwenhuizen-wijbenga,NaN,NaN,False,4,Dan ben ik erdoorheen.,NL-TweedeKamer,NLD,2019-05-22
1136587,2019-06-18,NaN,527,minister bruins,NaN,NaN,False,4,Dan ben ik erdoorheen.,NL-TweedeKamer,NLD,2019-06-18
1099595,2018-12-18,NaN,8,de heer van dam,CDA,1157.0,False,6,"Nee, maar ... ik ga weer.",NL-TweedeKamer,NLD,2018-12-18
935091,2016-05-25,NaN,524,de voorzitter,NaN,NaN,True,4,"Tot slot, mevrouw Gesthuizen.",NL-TweedeKamer,NLD,2016-05-25
935806,2016-05-26,NaN,463,de heer grashoff,GL,1537.0,False,4,Dan ga ik door.,NL-TweedeKamer,NLD,2016-05-26
1047877,2018-04-17,NaN,473,minister ollongren,NaN,NaN,False,10,"Ja, dan ga ik verder en dan rond ik af.",NL-TweedeKamer,NLD,2018-04-17
1117695,2019-03-13,NaN,946,de heer ronnes,CDA,1157.0,False,5,Ik ga zo meteen wel.,NL-TweedeKamer,NLD,2019-03-13
